In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install finance-datareader
!pip install neuralprophet[live]

     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 155 kB 10.9 MB/s 
  Attempting uninstall: holidays
    Found existing installation: holidays 0.10.5.2
    Uninstalling holidays-0.10.5.2:
      Successfully uninstalled holidays-0.10.5.2


In [3]:
import numpy as np
import pandas as pd
import gc
import math
import os.path
import time
import matplotlib.pyplot as plt
from datetime import timedelta, datetime
from dateutil import parser
from tqdm import tqdm
import copy
from fbprophet import Prophet

from neuralprophet import NeuralProphet
from neuralprophet import set_random_seed 

import warnings
import datetime
import random
import tensorflow as tf
warnings.filterwarnings("ignore")

# PUBLIC

In [4]:
path = '/content/drive/MyDrive/stock_2/'
list_name = 'stock_list.csv'
sample_name = 'sample_submission.csv'

stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI
...,...,...,...
365,맘스터치,220630,KOSDAQ
366,다날,064260,KOSDAQ
367,제이시스메디칼,287410,KOSDAQ
368,크리스에프앤씨,110790,KOSDAQ


In [5]:
import os
fileList = os.listdir('/content/drive/MyDrive/stock_2/train')

In [6]:
companyList = []
for file in fileList:
    companyName = file.split("_")[1]
    companyName = companyName.split('.')[0]
    if companyName != "csv":
        companyList.append(companyName)
companyList = sorted(companyList)
print(companyList)

['000060', '000080', '000100', '000120', '000150', '000240', '000250', '000270', '000660', '000670', '000720', '000810', '000880', '000990', '001230', '001440', '001450', '001740', '002380', '002790', '003000', '003090', '003380', '003410', '003490', '003670', '003800', '004000', '004020', '004170', '004370', '004490', '004800', '004990', '005250', '005290', '005300', '005380', '005385', '005387', '005490', '005830', '005850', '005930', '005935', '005940', '006260', '006280', '006360', '006400', '006650', '006730', '006800', '007070', '007310', '007390', '008560', '008770', '008930', '009150', '009240', '009540', '009830', '009900', '010060', '010120', '010130', '010620', '010780', '010950', '011000', '011070', '011170', '011200', '011210', '011780', '012330', '012450', '012510', '012750', '013120', '013890', '014680', '015750', '015760', '016360', '016380', '017670', '017800', '018260', '018880', '019170', '020150', '021240', '022100', '023530', '024110', '025900', '025980', '026960',

In [7]:
sample_submission = pd.read_csv(path + 'sample_submission.csv')
sample_submission = sample_submission[['Day']]
sample_submission['Day'] =pd.to_datetime(sample_submission['Day'])

In [8]:
for i in tqdm(sorted(companyList)):
    globals()['df_{}'.format(i)] = pd.read_csv('/content/drive/MyDrive/stock_2/train/stock_' + i + ".csv")
    locals()['df_{}'.format(i)] = locals()['df_{}'.format(i)][['Date', 'Close']]
    locals()['df_{}'.format(i)].rename(columns={'Date':'ds'}, inplace=True)
    locals()['df_{}'.format(i)].rename(columns={'Close':'y'}, inplace=True)

100%|██████████| 370/370 [01:36<00:00,  3.83it/s]


In [9]:
# model = NeuralProphet(
#     n_forecasts=5,
#         n_lags=370,
#         yearly_seasonality=False,
#         weekly_seasonality=False,
#         daily_seasonality=False,
#         batch_size=64,
#         epochs=100,
#         learning_rate=1.0,
# )

# metric = model.fit(df_000060, freq='D', valid_p=0.1, epochs=100)

# future = model.make_future_dataframe(df_000060, periods=7)
# forecast = model.predict(future)

In [ ]:
for i in sorted(companyList):
    globals()['model_{}'.format(i)] = NeuralProphet(
        n_forecasts=5,
        n_lags=5,
        yearly_seasonality='auto',
        weekly_seasonality='auto',
        daily_seasonality='auto',
        batch_size=64,
        epochs=100,
        learning_rate=0.1,
        loss_func='mae',
        normalize='auto'
        )

    globals()['metric_{}'.format(i)] = locals()['model_{}'.format(i)].fit(locals()['df_{}'.format(i)],
    freq='D',
    valid_p=0.1,
    epochs=100)

    globals()['future_{}'.format(i)] = locals()['model_{}'.format(i)].make_future_dataframe(locals()['df_{}'.format(i)], periods=5)
    globals()['forecast_{}'.format(i)] = locals()['model_{}'.format(i)].predict(locals()['future_{}'.format(i)])

In [14]:
for i in sorted(companyList):
    globals()['list_{}'.format(i)] = []
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat1'][5])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat2'][6])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat3'][7])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat4'][8])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat5'][9])

In [15]:
sample_submission = pd.read_csv(path + 'sample_submission.csv')
sub_public = sample_submission[0:5]
sub_private = sample_submission[5:10]

In [16]:
for i in sorted(companyList):
    sub_public[i] = locals()['list_{}'.format(i)]

In [18]:
import os
fileList = os.listdir('/content/drive/MyDrive/stock_2/test')

In [19]:
companyList = []
for file in fileList:
    companyName = file.split("_")[1]
    companyName = companyName.split('.')[0]
    if companyName != "csv":
        companyList.append(companyName)
companyList = sorted(companyList)
print(companyList)

['000060', '000080', '000100', '000120', '000150', '000240', '000250', '000270', '000660', '000670', '000720', '000810', '000880', '000990', '001230', '001440', '001450', '001740', '002380', '002790', '003000', '003090', '003380', '003410', '003490', '003670', '003800', '004000', '004020', '004170', '004370', '004490', '004800', '004990', '005250', '005290', '005300', '005380', '005385', '005387', '005490', '005830', '005850', '005930', '005935', '005940', '006260', '006280', '006360', '006400', '006650', '006730', '006800', '007070', '007310', '007390', '008560', '008770', '008930', '009150', '009240', '009540', '009830', '009900', '010060', '010120', '010130', '010620', '010780', '010950', '011000', '011070', '011170', '011200', '011210', '011780', '012330', '012450', '012510', '012750', '013120', '013890', '014680', '015750', '015760', '016360', '016380', '017670', '017800', '018260', '018880', '019170', '020150', '021240', '022100', '023530', '024110', '025900', '025980', '026960',

In [20]:
for i in tqdm(sorted(companyList)):
    globals()['df_{}'.format(i)] = pd.read_csv('/content/drive/MyDrive/stock_2/test/stock_' + i + ".csv")
    locals()['df_{}'.format(i)] = locals()['df_{}'.format(i)][['Date', 'Close']]
    locals()['df_{}'.format(i)].rename(columns={'Date':'ds'}, inplace=True)
    locals()['df_{}'.format(i)].rename(columns={'Close':'y'}, inplace=True)

100%|██████████| 370/370 [01:03<00:00,  5.84it/s]


In [21]:
for i in sorted(companyList):
    globals()['model_{}'.format(i)] = NeuralProphet(
        n_forecasts=5,
        n_lags=5,
        yearly_seasonality='auto',
        weekly_seasonality='auto',
        daily_seasonality='auto',
        batch_size=64,
        epochs=100,
        learning_rate=0.1,
        loss_func='mae',
        normalize='auto'
        )

    globals()['metric_{}'.format(i)] = locals()['model_{}'.format(i)].fit(locals()['df_{}'.format(i)],
    freq='D',
    valid_p=0.1,
    epochs=100)

    globals()['future_{}'.format(i)] = locals()['model_{}'.format(i)].make_future_dataframe(locals()['df_{}'.format(i)], periods=5)
    globals()['forecast_{}'.format(i)] = locals()['model_{}'.format(i)].predict(locals()['future_{}'.format(i)])

Epoch[100/100]: 100%|██████████| 100/100 [00:03<00:00, 27.33it/s, L1Loss=0.0532, MAE=1.44e+4, MSE=4.21e+8, RegLoss=0]
INFO - (NP.forecaster._handle_missing_data) - 103 missing dates added.
INFO - (NP.forecaster._handle_missing_data) - 103 NaN values in column y were auto-imputed.
INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
Epoch[100/100]: 100%|██████████| 100/100 [00:03<00:00, 28.94it/s, L1Loss=0.0363, MAE=1.31e+3, MSE=3.66e+6, RegLoss=0]
INFO - (NP.forecaster._handle_missing_data) - 103 missing dates added.
INFO - (NP.forecaster._handle_missing_data) - 103 NaN values in column y were auto-imputed.
INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
INFO - (NP.utils.set_auto_

In [23]:
for i in sorted(companyList):
    globals()['list_{}'.format(i)] = []
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat1'][5])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat2'][6])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat3'][7])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat4'][8])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat5'][9])

In [24]:
for i in sorted(companyList):
    sub_private[i] = locals()['list_{}'.format(i)]

In [25]:
sub_total = pd.concat([sub_public, sub_private])

In [26]:
sub_total

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,000670,000720,000810,000880,000990,001230,001440,001450,001740,002380,002790,003000,003090,003380,003410,003490,003670,003800,004000,004020,004170,004370,004490,004800,004990,005250,005290,005300,005380,005385,...,272290,273130,278280,278530,282330,285130,287410,290510,290650,292150,293490,293780,294090,294870,298000,298020,298050,298380,299030,299660,299900,307950,314130,316140,319400,319660,321550,323990,326030,330590,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,28203.527344,35008.214844,59739.609375,140992.656250,101415.296875,16725.107422,47269.261719,84998.281250,101151.328125,693409.3125,49526.941406,235585.375000,33670.625000,55526.078125,17002.869141,2261.340332,26960.812500,5068.477539,326457.37500,49911.187500,12591.780273,32107.353516,9564.023438,7676.483398,30492.410156,147039.734375,46665.480469,85053.335938,44086.117188,246155.718750,284649.34375,82279.710938,102521.710938,32363.816406,29096.300781,35774.148438,152772.218750,206803.421875,96939.617188,...,40231.242188,106682.273438,285754.81250,12691.437500,161111.906250,184843.156250,7005.878418,6919.686523,34293.445312,13377.663086,80074.453125,48854.636719,51613.449219,25254.078125,287794.09375,591127.12500,720013.0000,22272.064453,52820.542969,67272.921875,26410.025391,115511.734375,35398.164062,13029.406250,3380.744141,37984.925781,18220.375000,63393.468750,94242.679688,5664.490234,49057.667969,51664.816406,85541.218750,34322.734375,25728.650391,51774.203125,337297.87500,257795.234375,27166.142578,16998.585938
1,2021-11-02,28130.746094,35163.753906,59734.128906,140499.781250,100786.796875,16572.847656,46720.210938,84764.406250,99982.296875,695317.3750,49737.019531,233578.437500,33486.195312,55317.687500,16779.250000,2240.058350,26770.335938,5065.397949,328349.25000,49765.812500,12579.625000,32051.123047,9529.366211,7651.786621,30437.769531,147347.375000,46679.050781,85475.312500,43969.710938,245518.390625,284150.37500,82465.210938,101242.757812,32354.478516,28834.007812,35505.789062,150159.015625,206201.750000,96839.203125,...,40867.617188,106541.343750,285973.28125,12662.767578,160646.000000,184055.156250,7038.096680,6910.311035,34299.445312,13297.677734,80281.226562,47274.343750,51658.636719,25175.693359,285939.03125,587247.12500,717621.5000,21075.392578,52792.175781,67835.703125,26228.720703,115702.250000,35622.765625,12860.052734,3355.986328,37674.929688,18257.716797,64036.027344,92053.265625,5657.022461,49384.277344,50293.875000,85647.531250,34043.765625,27163.984375,51426.007812,332964.25000,257220.328125,27038.677734,16856.957031
2,2021-11-03,28049.812500,35311.929688,59844.972656,141750.828125,100683.562500,16611.242188,47544.941406,84865.617188,101272.906250,686279.2500,49830.187500,243394.843750,33811.320312,55183.371094,16760.003906,2228.275879,27309.234375,5056.190430,327134.03125,50234.148438,12613.612305,32258.453125,9497.806641,7728.198242,30528.681641,146388.375000,46814.761719,84939.117188,43908.566406,246783.531250,284152.71875,82123.921875,100747.773438,32455.486328,28856.291016,35483.640625,153536.828125,206519.078125,97068.804688,...,41892.976562,106297.210938,284717.21875,12693.712891,162361.562500,184094.750000,6990.015625,6940.990234,34108.187500,13200.173828,80754.859375,46489.503906,52125.394531,24944.865234,282731.87500,581164.37500,708790.8125,20954.308594,52435.160156,67257.242188,27176.273438,115520.343750,36886.812500,13023.773438,3451.913086,37431.339844,18132.828125,63803.332031,94035.312500,5662.452148,48781.957031,51497.750000,85727.460938,33629.648438,26394.373047,51925.781250,341824.50000,258397.609375,27054.203125,16728.011719
3,2021-11-04,28585.082031,35309.554688,59785.378906,142122.234375,105060.281250,16849.525391,47406.074219,84485.140625,102192.656250,698654.3125,50056.167969,251522.375000,33864.511719,54872.738281,17012.542969,2206.057617,27716.476562,5051.872559,323576.06250,50128.367188,12829.621094,32160.82

In [27]:
sub_total.to_csv(path + 'neural_prophet_lag_5.csv', index=False)